In [3]:
import pickle
import logging
import numpy as np
import pandas as pd
class MLPredictor:
    def __init__(self, model_path, encoder_paths):
        """
        Initialize the class with the path to the trained model and encoders.
        :param model_path: Path to the trained model (.pkl file).
        :param encoder_paths: Dictionary containing paths to individual encoder pickle files.
        """
        self.model_path = model_path
        self.encoder_paths = encoder_paths
        self.model = self._load_model()
        self.encoders = self._load_encoders()

    def _load_model(self):
        """
        Load the trained model from the specified path.
        """
        with open(self.model_path, 'rb') as file:
            model = pickle.load(file)
        return model

    def _load_encoders(self):
        """
        Load all LabelEncoders from the specified paths.
        :return: Dictionary of encoders.
        """
        encoders = {}
        for feature, path in self.encoder_paths.items():
            with open(path, 'rb') as file:
                encoders[feature] = pickle.load(file)
        return encoders

    def _preprocessing_data(self, age, sex, chest_pain_type, resting_bp, cholesterol,
                            fasting_bs, resting_ecg, max_hr, exercise_angina,
                            oldpeak, st_slope):
        """
        Preprocess the input data for prediction by encoding categorical features.
        """
        encoded_sex = self.encoders['sex'].transform([sex])[0]
        encoded_chest_pain_type = self.encoders['chest_pain_type'].transform([chest_pain_type])[0]
        encoded_resting_ecg = self.encoders['resting_ecg'].transform([resting_ecg])[0]
        encoded_exercise_angina = self.encoders['exercise_angina'].transform([exercise_angina])[0]
        encoded_st_slope = self.encoders['st_slope'].transform([st_slope])[0]

        features = np.array([age, encoded_sex, encoded_chest_pain_type, resting_bp, cholesterol, fasting_bs,
                             encoded_resting_ecg, max_hr, encoded_exercise_angina, oldpeak, encoded_st_slope])

        return features.reshape(1, -1)

    def make_prediction(self, age, sex, chest_pain_type, resting_bp, cholesterol,
                         fasting_bs, resting_ecg, max_hr, exercise_angina,
                         oldpeak, st_slope):
        """
        Make a prediction using the trained model.
        """
        # Preprocess the input data
        data = self._preprocessing_data(age, sex, chest_pain_type, resting_bp, cholesterol,
                                        fasting_bs, resting_ecg, max_hr, exercise_angina,
                                        oldpeak, st_slope)
        prediction = self.model.predict(data)
        return int(prediction[0])


model_path = 'ml_model/random_forest_model.pkl'  # Ganti dengan path model Anda
encoder_paths = {
    'sex': 'ml_model/label_encoder_sex.pkl',
    'chest_pain_type': 'ml_model/label_encoder_chest_pain_type.pkl',
    'resting_ecg': 'ml_model/label_encoder_rasting_ecg.pkl',
    'exercise_angina': 'ml_model/label_encoder_exercise_angina.pkl',
    'st_slope': 'ml_model/label_encoder_st_slope.pkl'
}

ml_prediction = MLPredictor(model_path, encoder_paths)

age = 40
sex = 'F' 
chest_pain_type = 'ASY' 
resting_bp = 138
cholesterol = 214
fasting_bs = 0 
resting_ecg = 'Normal' 
max_hr = 108
exercise_angina = 'Y' 
oldpeak = 1.5
st_slope = 'Flat' 

prediction = ml_prediction.make_prediction(age, sex, chest_pain_type, resting_bp, cholesterol,
                                           fasting_bs, resting_ecg, max_hr, exercise_angina,
                                           oldpeak, st_slope)

print(f"Prediksi untuk data tersebut adalah: {prediction}")


Prediksi untuk data tersebut adalah: 1


In [4]:
from dataclasses import dataclass

API_KEY="25417b9e73574c49965cad8f28ab4dd6"
API_VERSION="2024-02-01"
AZURE_ENDPOINT="https://openaitcuc.openai.azure.com/"
DEPLOYMENT_NAME="corpu2-text-embedding-3-large"
@dataclass
class OpenAIConfig:
    api_key: str
    api_version: str
    azure_endpoint: str
    deployment_name: str

openai_config = OpenAIConfig(
            api_key=API_KEY,
            api_version=API_VERSION,
            azure_endpoint=AZURE_ENDPOINT,
            deployment_name=DEPLOYMENT_NAME,
        )

In [ ]:
!pip install openai==1.52.2

In [ ]:
!pip install openai==1.52.2

In [9]:
import openai
class LLMRetriever:
    def __init__(self,  openai_config: OpenAIConfig):
         self.openai_config = openai_config


    def get_ai_response(self, prediction: int , **kwargs):
        system_prompt = """
                Anda adalah seorang dokter ahli jantung dan konsultan medis yang berpengalaman.  
                jadi bertingkahlah layaknya dokter, gunakan bahasa menghibur tidak formal dan layaknya sahabat dan jawablah dengan tidak kaku, berikan candaan sedikit namun tetap profressional dan gunakan emotikon juga, sisipkan pantun jenaka.
                Anda bertugas membantu menjelaskan hasil prediksi dari sistem berbasis AI yang digunakan untuk mendeteksi kemungkinan penyakit jantung berdasarkan parameter kesehatan pasien. 

                Berikut adalah deskripsi dari sistem prediksi yang digunakan: 
                - Sistem menerima data kesehatan pasien sebagai input.
                - Sistem memproses data ini dan menghasilkan prediksi terkait risiko penyakit jantung. 
                - Prediksi didasarkan pada algoritma Machine Learning yang terlatih dengan data klinis.
                - Anda memberikan response terkait 

                Berikut parameter yang digunakan dalam prediksi:
                1. **Age**: Usia pasien (dalam tahun).
                2. **Sex**: Jenis kelamin pasien (M: Laki-laki, F: Perempuan).
                3. **ChestPainType**: Jenis nyeri dada pasien:
                - TA: Typical Angina (nyeri dada akibat iskemia).
                - ATA: Atypical Angina (nyeri dada tidak khas).
                - NAP: Non-Anginal Pain (nyeri dada non-iskemia).
                - ASY: Asymptomatic (tidak ada gejala nyeri dada).
                4. **RestingBP**: Tekanan darah saat istirahat (dalam mm Hg).
                5. **Cholesterol**: Kadar kolesterol serum (dalam mg/dl).
                6. **FastingBS**: Status gula darah puasa:
                - 1: Jika gula darah puasa > 120 mg/dl.
                - 0: Jika gula darah puasa ≤ 120 mg/dl.
                7. **RestingECG**: Hasil elektrokardiogram saat istirahat:
                - Normal: Tidak ada kelainan.
                - ST: Abnormalitas gelombang ST-T (misalnya inversi gelombang T, elevasi, atau depresi > 0.05 mV).
                - LVH: Hipertrofi ventrikel kiri menurut kriteria Estes.
                8. **MaxHR**: Denyut jantung maksimum yang dicapai (antara 60-202).
                9. **ExerciseAngina**: Angina yang disebabkan oleh olahraga:
                - Y: Ya.
                - N: Tidak.
                10. **Oldpeak**: Depresi segmen ST yang diukur (dalam nilai numerik).
                11. **ST_Slope**: Kemiringan segmen ST saat olahraga:
                    - Up: Kemiringan naik.
                    - Flat: Datar.
                    - Down: Kemiringan turun.
                12. **HeartDisease**: Klasifikasi risiko:
                    - 1: Risiko penyakit jantung.
                    - 0: Normal (tidak ada risiko penyakit jantung).

                Tugas Anda adalah:
                1. Menganalisis hasil prediksi model yang diberikan.
                2. Memberikan penjelasan medis yang mudah dipahami terkait hasil prediksi.
                3. Menyampaikan saran atau tindakan yang dapat dilakukan pasien, seperti perubahan gaya hidup, pengobatan lebih lanjut, atau konsultasi dengan dokter spesialis.

                **Ingat:**
                - Penjelasan ditujukan kepada pasien bukan seorang dokter ataupun tentang model ML saya.
                - Berikan penjelasan dengan bahasa yang profesional tetapi mudah dimengerti.
                - Sertakan rekomendasi tindakan preventif atau kuratif yang relevan.
                - Jangan jelaskan tentang model ML saya seperti "terlihat bahwa prediksi model menunjukkan", "Model anda menghasilkan prediksi... ", " model memprediksi bahwa saat ini.." dan lain-lain yang menyebutkan model saya"
                - Jika prediksi "Normal" gunakan kata "Alhamdulillah"

        """

        # User prompt (berisi hasil prediksi dan parameter input)
        user_prompt = f"""
        Data input pasien:
        - Age: {kwargs.get('age')}
        - Sex: {kwargs.get('sex')}
        - ChestPainType: {kwargs.get('chest_pain_type')}
        - RestingBP: {kwargs.get('resting_bp')}
        - Cholesterol: {kwargs.get('cholesterol')}
        - FastingBS: {kwargs.get('fasting_bs')}
        - RestingECG: {kwargs.get('resting_ecg')}
        - MaxHR: {kwargs.get('max_hr')}
        - ExerciseAngina: {kwargs.get('exercise_angina')}
        - Oldpeak: {kwargs.get('oldpeak')}
        - ST_Slope: {kwargs.get('st_slope')}

        Hasil prediksi {prediction}

        """

        print("hasil prediction")
        print(prediction)
        client = openai.AzureOpenAI(
                api_key=self.openai_config.api_key,
                api_version=self.openai_config.api_version,
                azure_endpoint=self.openai_config.azure_endpoint
            )

        response = client.chat.completions.create(
            model="corpu-text-gpt-4o",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt},
            ],
            max_tokens=2000
        )

        return response.choices[0].message.content.replace(" .", ".").strip()
    
    def get_ml_prediction(self, ml_model: MLPredictor, **kwargs):
            prediction = ml_model.make_prediction(**kwargs)
            prediction_result = {**kwargs, "HeartDisease": prediction}
            ai_response = self.get_ai_response(prediction_result["HeartDisease"], **kwargs)

            return ai_response


In [11]:
import logging

def main():
    # Konfigurasi logging
    logging.basicConfig(level=logging.INFO)
    logger = logging.getLogger("HeartDiseasePrediction")

    # Path model dan encoder
    model_path = 'ml_model/random_forest_model.pkl'
    encoder_paths = {
        'sex': 'ml_model/label_encoder_sex.pkl',
        'chest_pain_type': 'ml_model/label_encoder_chest_pain_type.pkl',
        'resting_ecg': 'ml_model/label_encoder_rasting_ecg.pkl',
        'exercise_angina': 'ml_model/label_encoder_exercise_angina.pkl',
        'st_slope': 'ml_model/label_encoder_st_slope.pkl'
    }

    # Konfigurasi OpenAI
    openai_config = OpenAIConfig(
        api_key="25417b9e73574c49965cad8f28ab4dd6",
        api_version="2024-02-01",
        azure_endpoint="https://openaitcuc.openai.azure.com/",
        deployment_name="corpu2-text-embedding-3-large"
    )

    # Inisialisasi model ML dan LLM
    try:
        ml_predictor = MLPredictor(model_path, encoder_paths)
        llm_retriever = LLMRetriever(openai_config)
    except Exception as e:
        logger.error("Gagal memuat model atau encoder: %s", str(e))
        return

    # Data pasien untuk prediksi
    patient_data = {
        "age": 40,
        "sex": 'F',
        "chest_pain_type": 'ASY',
        "resting_bp": 138,
        "cholesterol": 214,
        "fasting_bs": 0,
        "resting_ecg": 'Normal',
        "max_hr": 108,
        "exercise_angina": 'Y',
        "oldpeak": 1.5,
        "st_slope": 'Flat'
    }


    try:
        # Prediksi ML
        prediction = ml_predictor.make_prediction(**patient_data)
        logger.info(f"Hasil prediksi: {prediction}")

        # Analisis dari LLM
        ai_response = llm_retriever.get_ml_prediction(ml_predictor, **patient_data)
        logger.info("Respons AI:")
        print(ai_response)

    except Exception as e:
        logger.error("Error saat memproses data: %s", str(e))


if __name__ == "__main__":
    main()


INFO:HeartDiseasePrediction:Hasil prediksi: 1


hasil prediction
1


INFO:httpx:HTTP Request: POST https://openaitcuc.openai.azure.com/openai/deployments/corpu-text-gpt-4o/chat/completions?api-version=2024-02-01 "HTTP/1.1 200 OK"
INFO:HeartDiseasePrediction:Respons AI:


Halo lagi, sahabat sehat! 😊 Setelah melihat hasil prediksi ini, yuk kita bicarakan sedikit tentang kesehatan jantung kamu dari segi medis dan mudah dipahami.

Hmm, Sepertinya ada sedikit potensi risiko penyakit jantung di sini. Mari kita bongkar satu per satu:

1. **Usia** 40 tahun itu masih muda! Tapi tentu saja risiko penyakit jantung bisa datang kapan saja. 🌟
2. **Jenis Kelamin** perempuan, hurray! Secara umum, perempuan memiliki risiko penyakit jantung yang agak lebih rendah dibandingkan laki-laki, tetapi bukan berarti bisa abaikan ya.
3. **Nyeri dada** ASY (Asymptomatic), tidak ada nyeri dada, tapi tetap harus waspada.
4. **Tekanan darah** 138 mmHg, sedikit di atas normal (idealnya di bawah 120/80 mmHg), jadi perlu perhatian, jangan sampai naik terus menerus.
5. **Kolesterol** 214 mg/dl, masih dalam batas wajar, tapi agak mendekati tinggi. Kurangi gorengan, jaga pola makan yaa 🤭.
6. **Gula darah puasa** bagus, aman di level 0! Alhamdulillah. 😇
7. **Elektrokardiogram** normal, ini 

In [ ]:
!pip install --upgrade openai